In [265]:
import json
from statsmodels.tsa.exponential_smoothing.ets import ETSModel
import pandas as pd

def __forecast_ets_model(history_df, history_days):
    history_df = history_df.set_index('date')
    shape_0 = history_df.shape[0]
    data_train = history_df
    if (shape_0 >= 60):
        data_train = history_df.iloc[-history_days:, :]
    elif (shape_0 > 7):
        data_train = history_df.iloc[-min(shape_0, history_days):, :]

    seasonal = 'mul'
    seasonal_periods = 7
    if (shape_0 <= 7 or history_days <= 7):
        seasonal = None
        seasonal_periods = None
    data_train = pd.Series(data_train['sales']).astype('float64')
    ets_model_holdout = ETSModel(data_train, error='mul', trend='add', seasonal=seasonal,
                                 damped_trend=True, seasonal_periods=seasonal_periods,
                                 initial_level=data_train.values.mean(), freq='D')
    ets_fitted_holdout = ets_model_holdout.fit()

    return ets_fitted_holdout

copyright()

Copyright (c) 2001-2019 Python Software Foundation.
All Rights Reserved.

Copyright (c) 2000 BeOpen.com.
All Rights Reserved.

Copyright (c) 1995-2001 Corporation for National Research Initiatives.
All Rights Reserved.

Copyright (c) 1991-1995 Stichting Mathematisch Centrum, Amsterdam.
All Rights Reserved.


In [266]:
import pandas as pd
sales_data = pd.read_excel('milk-day-sales.xlsx')
date_series =  pd.Series(sales_data['date'])
sales_data['date'] = pd.date_range(start=date_series[0] ,periods=len(date_series), freq='D').strftime('%Y-%m-%d')
# sales_data = sales_data.set_index('date')
# sales_data.head(), sales_data.tail()
# date_series[0],date_series[len(date_series)-1]

In [267]:
json_str = sales_data.to_json(orient="records")
print(json_str)

[{"date":"2021-05-13","sales":676},{"date":"2021-05-14","sales":667},{"date":"2021-05-15","sales":372},{"date":"2021-05-16","sales":259},{"date":"2021-05-17","sales":270},{"date":"2021-05-18","sales":139},{"date":"2021-05-19","sales":298},{"date":"2021-05-20","sales":304},{"date":"2021-05-21","sales":243},{"date":"2021-05-22","sales":293},{"date":"2021-05-23","sales":227},{"date":"2021-05-24","sales":103},{"date":"2021-05-25","sales":428},{"date":"2021-05-26","sales":462},{"date":"2021-05-27","sales":309},{"date":"2021-05-28","sales":374},{"date":"2021-05-29","sales":196},{"date":"2021-05-30","sales":266},{"date":"2021-05-31","sales":163},{"date":"2021-06-01","sales":317},{"date":"2021-06-02","sales":283},{"date":"2021-06-03","sales":358},{"date":"2021-06-04","sales":418},{"date":"2021-06-05","sales":450},{"date":"2021-06-06","sales":496},{"date":"2021-06-07","sales":349},{"date":"2021-06-08","sales":490},{"date":"2021-06-09","sales":485},{"date":"2021-06-10","sales":440},{"date":"2021

In [268]:
json_str = '''
[
{"date":1621209600000,"sales":270},{"date":1621296000000,"sales":139},{"date":1621382400000,"sales":298},{"date":1621468800000,"sales":304},
{"date":1620864000000,"sales":676},{"date":1620950400000,"sales":667},{"date":1621036800000,"sales":372},{"date":1621123200000,"sales":259}
]
'''
json_data = pd.read_json(json_str)
json_data = json_data.set_index('date')
json_data = json_data.sort_index()
# json_data.index = json_data['date']
[].sort(reverse=True)
json_data

,sales
date,
2021-05-13,676
2021-05-14,667
2021-05-15,372
2021-05-16,259
2021-05-17,270
2021-05-18,139
2021-05-19,298
2021-05-20,304


In [269]:
len(json_data.shape)
json.dumps(json_data.shape)
json_data.shape[0]

8

In [270]:
from datetime import datetime

history_60_ets_model = __forecast_ets_model(sales_data, 60)
history_7_ets_model = __forecast_ets_model(sales_data, 7)
start = '2021-10-10'
end = '2021-10-31'
forecast_df = pd.DataFrame({
                             'his_60': history_60_ets_model.predict(start=start,end=end),
                             'his_7': history_7_ets_model.predict(start=start,end=end) },
                           index = pd.date_range(start=start,end=end, name='datetime'))

forecast_df['forecast_float'] = forecast_df['his_60']*0.4 + forecast_df['his_7']*0.6

forecast_df['date'] = pd.date_range(start=start,end=end).strftime('%Y-%m-%d')
forecast_df['forecast']  = pd.Series(forecast_df['forecast_float']).astype('int')
# forecast_df['forecast'][0],forecast_df['forecast'][-1]
result = forecast_df.loc[:,['date','forecast']]
# result = result.set_index('date')
forecast_df,\
result
# print(result)

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           12     M =           10

At X0         1 variables are exactly at the bounds

At iterate    0    f=  7.26317D+00    |proj g|=  1.40328D+00

At iterate    1    f=  7.11471D+00    |proj g|=  1.04463D+00

At iterate    2    f=  6.92348D+00    |proj g|=  8.93253D-01

At iterate    3    f=  6.79128D+00    |proj g|=  6.81952D-01

At iterate    4    f=  6.71114D+00    |proj g|=  3.34639D-01

At iterate    5    f=  6.66681D+00    |proj g|=  1.70771D-01

At iterate    6    f=  6.64758D+00    |proj g|=  6.08481D-02

At iterate    7    f=  6.63932D+00    |proj g|=  4.97184D-02

At iterate    8    f=  6.63345D+00    |proj g|=  6.06942D-02

At iterate    9    f=  6.61945D+00    |proj g|=  1.21007D-01

At iterate   10    f=  6.61053D+00    |proj g|=  8.34738D-02

At iterate   11    f=  6.60456D+00    |proj g|=  2.89996D-02

At iterate   12    f=  6.60441D+00    |proj g|=  2.82722D-02

At iterate   13    f=  6.6

(                his_60       his_7  forecast_float        date  forecast
 datetime                                                                
 2021-10-10  322.952916  487.177069      421.487408  2021-10-10       421
 2021-10-11  336.264726  519.763310      446.363876  2021-10-11       446
 2021-10-12  269.280675  545.842396      435.217708  2021-10-12       435
 2021-10-13  438.527572  566.705665      515.434428  2021-10-13       515
 2021-10-14  384.031294  583.396281      503.650286  2021-10-14       503
 2021-10-15  427.671438  596.748773      529.117839  2021-10-15       529
 2021-10-16  352.117290  607.430767      505.305376  2021-10-16       505
 2021-10-17  343.203402  615.976362      506.867178  2021-10-17       506
 2021-10-18  336.255334  622.812838      508.189836  2021-10-18       508
 2021-10-19  269.274185  628.282018      484.678885  2021-10-19       484
 2021-10-20  438.518452  632.657363      555.001799  2021-10-20       555
 2021-10-21  384.024402  636.157639   

In [271]:
json_str = result.to_json(orient="records")
json_str

'[{"date":"2021-10-10","forecast":421},{"date":"2021-10-11","forecast":446},{"date":"2021-10-12","forecast":435},{"date":"2021-10-13","forecast":515},{"date":"2021-10-14","forecast":503},{"date":"2021-10-15","forecast":529},{"date":"2021-10-16","forecast":505},{"date":"2021-10-17","forecast":506},{"date":"2021-10-18","forecast":508},{"date":"2021-10-19","forecast":484},{"date":"2021-10-20","forecast":555},{"date":"2021-10-21","forecast":535},{"date":"2021-10-22","forecast":554},{"date":"2021-10-23","forecast":525},{"date":"2021-10-24","forecast":523},{"date":"2021-10-25","forecast":521},{"date":"2021-10-26","forecast":495},{"date":"2021-10-27","forecast":563},{"date":"2021-10-28","forecast":541},{"date":"2021-10-29","forecast":559},{"date":"2021-10-30","forecast":529},{"date":"2021-10-31","forecast":526}]'